In [1]:
import pymysql
# 네트워크 접속은 os 관리
conn = pymysql.connect(host = '192.168.1.242', 
                     user='root', port=3306, password='q', db='acomschool',
                     charset='utf8mb4', use_unicode=True,
                     cursorclass=pymysql.cursors.DictCursor)

cur = conn.cursor()

cur.execute("""CREATE TABLE pet (
            id INT NOT NULL AUTO_INCREMENT,
           name VARCHAR(20), owner VARCHAR(20), species VARCHAR(20),
           sex CHAR(1), birth DATE, death DATE,
           PRIMARY KEY (id)) ENGINE=InnoDB AUTO_INCREMENT=1""")
print(cur.description)
conn.close()

In [5]:
conn = pymysql.connect(host = '192.168.1.242', 
                     user='root', port=3306, password='q', db='acomschool',
                     charset='utf8mb4', use_unicode=True,
                     cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()

# 데이터를 입력해봅시다
sql = "INSERT INTO pet (name, owner, species, sex, birth, death) VALUES (%s, %s, %s, %s, %s, %s)" 
cur.execute(sql, ('유리', 'acorn', '고양이', 'f', '2014-1-1', '2016-2-1'))
cur.execute(sql, ('귀염', 'acorn', '강아지', 'f', '2014-3-1', '2016-9-1'))
cur.execute(sql, ('돌이', 'acorn', '곰', 'f', '2014-8-1', '2018-2-1'))
conn.commit()

cur.executemany( # DB : NULL = PYthon : None
    "INSERT INTO pet(name, owner, species, sex, birth, death) VALUES (%s, %s, %s, %s, %s, %s)",
[('대한','민주','cat', 'f', '2001-02-04', None),
('민국','자유','cat','f','2010-03-17', None),
('만세','민주','cat','f','2010-05-13', None),
('하자','민주','cat','m','2015-08-27', None),
('성공','자유','cat','m','2017-08-31', '2018-04-29'),
])
conn.commit()
print(cur.rowcount)
conn.close()

5


In [13]:
conn = pymysql.connect(host = '192.168.1.242', 
                     user='root', port=3306, password='q', db='acomschool',
                     charset='utf8mb4', use_unicode=True,
                     cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()
cur.execute('select * from pet') # recordSet 생성
print(cur.rowcount) #8
cur.description # 설명
# 이번에는 데이터를 가져오자

cur.fetchone()
cur.fetchone()
cur.fetchone()
recs = cur.fetchmany(2)
for rec in recs:
    print(rec)
recs = cur.fetchall()
for rec in recs:
    print(rec)
cur.close()
conn.close()

8
{'id': 4, 'name': '대한', 'owner': '민주', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2001, 2, 4), 'death': None}
{'id': 5, 'name': '민국', 'owner': '자유', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2010, 3, 17), 'death': None}
{'id': 6, 'name': '만세', 'owner': '민주', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2010, 5, 13), 'death': None}
{'id': 7, 'name': '하자', 'owner': '민주', 'species': 'cat', 'sex': 'm', 'birth': datetime.date(2015, 8, 27), 'death': None}
{'id': 8, 'name': '성공', 'owner': '자유', 'species': 'cat', 'sex': 'm', 'birth': datetime.date(2017, 8, 31), 'death': datetime.date(2018, 4, 29)}


In [22]:
# 문제 : 생일이 2015-01-01 이후인 pet을 출력하시오

conn = pymysql.connect(host = '192.168.1.242', 
                     user='root', port=3306, password='q', db='acomschool',
                     charset='utf8mb4', use_unicode=True,
                     cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()
cur.execute("""select * from pet where birth > '2015-01-01';""")
rows = cur.fetchall()
for pet in rows:
    pet_val = list(pet.values())
    print("ID=%d, Name=%s, 주인=%s" % (pet_val[0], pet_val[1], pet_val[2]))

# 문제 : 유리를 초심으로 업데이트 하시오    
sql = 'update pet set name=%s where name=%s'
cur.execute(sql, ('초심', '유리'))
conn.commit()
print(cur.rowcount)

# 문제 : 초심을 삭제하시오 (쿼리를 일반화)
sql = 'delete from pet where name=%s'
cur.execute(sql, '초심')
conn.commit()
print(cur.rowcount)

cur.close()
conn.close()

ID=7, Name=하자, 주인=민주
ID=8, Name=성공, 주인=자유
1
1


In [24]:
# 저장 루틴 다루기
conn = pymysql.connect(host = '192.168.1.242', 
                     user='root', port=3306, password='q', db='acomschool',
                     charset='utf8mb4', use_unicode=True,
                     cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()

cur.callproc('pet_select')
if(cur.rowcount):
    print(cur.fetchall())
cur.close()
conn.close()

[{'id': 2, 'name': '귀염', 'owner': 'acorn', 'species': '강아지', 'sex': 'f', 'birth': datetime.date(2014, 3, 1), 'death': datetime.date(2016, 9, 1)}, {'id': 3, 'name': '돌이', 'owner': 'acorn', 'species': '곰', 'sex': 'f', 'birth': datetime.date(2014, 8, 1), 'death': datetime.date(2018, 2, 1)}, {'id': 4, 'name': '대한', 'owner': '민주', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2001, 2, 4), 'death': None}, {'id': 5, 'name': '민국', 'owner': '자유', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2010, 3, 17), 'death': None}, {'id': 6, 'name': '만세', 'owner': '민주', 'species': 'cat', 'sex': 'f', 'birth': datetime.date(2010, 5, 13), 'death': None}, {'id': 7, 'name': '하자', 'owner': '민주', 'species': 'cat', 'sex': 'm', 'birth': datetime.date(2015, 8, 27), 'death': None}, {'id': 8, 'name': '성공', 'owner': '자유', 'species': 'cat', 'sex': 'm', 'birth': datetime.date(2017, 8, 31), 'death': datetime.date(2018, 4, 29)}]


In [30]:
# 문제 : insert 를 하는 프로시져를 생성한 다음
# ('마음이', 'acorn', '백구', 'f', '2018-01-02', '2018-11-04') 데이터를 입력하시오
conn = pymysql.connect(host = '192.168.1.242', 
                     user='root', port=3306, password='q', db='acomschool',
                     charset='utf8mb4', use_unicode=True,
                     cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()

# heidi sql에서 저장 프로시저 생성 (insert는 매개변수 만들어줘야함)
# 쿼리문에서 첫번째는 필드명, 두번째는 변수명을 의미

data = ('마음이', 'acorn', '백구', 'f', '2018-01-02', '2018-11-04')
cur.callproc('pet_insert', data)
conn.commit()
cur.close()
conn.close()

In [40]:
# 문제 : 이름을 update 하는 프로시져를 작성하고
# 마음이를 마음으로 수정하시오

conn = pymysql.connect(host = '192.168.1.242', 
                     user='root', port=3306, password='q', db='acomschool',
                     charset='utf8mb4', use_unicode=True,
                     cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()

name_to_change = ('마음', '마음이',0)
cur.callproc('pet_updatename', name_to_change)
conn.commit()
cur.execute('SELECT @_pet_updatename_2') # 이름이 결정, 내부적으로 만들어줌 >> 리턴값 받아올 수 있음
result = cur.fetchone()
print(result['@_pet_updatename_2'])
if result['@_pet_updatename_2'] == 2:
    print("수정할 이름이 없습니다")
else:
    print("성공적으로 수정되었습니다.")

cur.close()
conn.close()

2
수정할 이름이 없습니다


In [42]:
# 삭제삭제

conn = pymysql.connect(host = '192.168.1.242', 
                     user='root', port=3306, password='q', db='acomschool',
                     charset='utf8mb4', use_unicode=True,
                     cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()
data = ('귀염',0)
cur.callproc('pet_delete', data)
conn.commit()
cur.execute('SELECT @_pet_delete_1')
result = cur.fetchone()
print(result['@_pet_delete_1'])
if result['@_pet_delete_1'] == 2:
    print("삭제할 이름이 없습니다.")
else:
    print("성공적으로 삭제하였습니다.")
cur.close()
conn.close() 
    

2
삭제할 이름이 없습니다.


In [44]:
# 어제는 서버쪽에서 rollback 하는 거 했고
# 오늘은 클라이언트측에서 rollback 하는 걸 해보겠음

conn = pymysql.connect(host = '192.168.1.242', 
                     user='root', port=3306, password='q', db='acomschool',
                     charset='utf8mb4', use_unicode=True,
                     cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()

try: # exception 예외처리 (프로그램은 지속적으로 실행되어야 하기 때문에)
    data = ("귀염", 0)
    cur.callproc('pet_delete', data)
    conn.commit()
    cur.execute('SELECT @_pet_delete_1')
    result = cur.fetchone()
    print(result['@_pet_delete_1'])
    if result['@_pet_delete_1'] == 2:
        print("삭제할 이름이 없습니다.")
    else:
        print("성공적으로 삭제하였습니다.")
except:
    conn.rollback()
cur.close()
conn.close()

2
삭제할 이름이 없습니다.


In [1]:
import pymysql

class Student:
    def __init__(self):
        self.bunho = 0
        self.name = ''
        self.kor = 0
        self.mat = 0
        self.eng = 0
        self.total = 0
        self.average = 0
        self.grade = ''
        
    def inputData(self):
        self.name = input("이름을 입력하세요")
        self.kor = eval(input("국어 점수:"))
        self.mat = eval(input("수학 점수:"))
        self.eng = eval(input("영어 점수:"))
        
    def calc_total_average(self):
        self.total = self.kor + self.mat + self.eng
        self.average = round(self.total / 3, 2)
        self.grade = self.calcSemGrade()
        
    def calcSemGrade(self):
        if self.average >= 90:
            return 'A'
        elif self.average >= 80:
            return 'B'
        elif self.average >= 70:
            return 'C'
        elif self.average >= 60:
            return 'D'
        else:
            return 'F'
        
    def __str__(self):
        return '%5s %5s %6s %6s %6s %7s %7s %s' % (self.bunho, self.name, self.kor,
                                                  self.mat, self.eng, self.total, self.average, self.grade)
    
    def __cmp__(self, other):
        return self.name == other.name
    
    def __call__(self, other): # 클래스를 함수처럼 사용하고자 할 때 오버라이딩
        st = []
        for student in other:
            stp = Student()
            stp.name = student[0]
            stp.kor = student[1]
            stp.mat = student[2]
            stp.eng = student[3]
            st.append(stp)
        return st

In [2]:
class maria:
    schoolname = "에이콘 아카데미"
    def __init__(self):
        self.conn = pymysql.connect(host='192.168.1.242', port=3306, user='root',
                           password='q',db='acomschool', charset='utf8',
                                    autocommit=True)
        cor = self.conn.cursor()
        cor.execute("SHOW TABLES")
        tables = cor.fetchall()
        c = []
        for i in tables:
            if  i == ('sungjuk',):
                c.append(i)
        if len(c) != 0:
            sungjuk_table = "SELECT * FROM sungjuk"
            cor.execute(sungjuk_table)
            print('테이블이 있습니다. ')
        else:
            cor.execute("CREATE TABLE sungjuk (bubho INTEGER(11) AUTO_INCREMENT, name VARCHAR(20), kor INTEGER(20),  mat INTEGER(20),eng INTEGER(20))")
            print('sungjuk테이블을 생성하였습니다. ') # 여기 번호가 없음! 만들어주고  auto_incre & pk 설정
    def __del__(self):
        self.conn.close()

    def selectdb(self ):
        cur = self.conn.cursor()
        cur.callproc("sungjuk_select")
        if (cur.rowcount):
            stu = list(cur.fetchall())
        else :
            stu = 0;
        cur.close()
        return stu
   
    def insertdb(self,name, kor, mat, eng):
        cur = self.conn.cursor()
        args = (name, kor, mat, eng)
        cur.callproc("sungjuk_insert",args)
        cur.close()
       
    def updatedb(self, in_name, up_name):
        cur = self.conn.cursor()
        args = (in_name, up_name, 0)
        cur.callproc("sungjuk_update", args)
        cur.execute('SELECT @_sungjuk_update_2')
        result = cur.fetchone()
        if result==2 :
            print("수정할 이름이 없습니다.")
        else :
            print("성공적으로 수정하였습니다. ")
        cur.close()
        return result 
   
    def deletedb(self,in_name):
        cur = self.conn.cursor()
        args = (in_name, 0)
        cur.callproc("sungjuk_delete", args)
        cur.execute('SELECT @_sungjuk_delete_1')
        result = cur.fetchone()
        if result==2 :
            print("삭제할 이름이 없습니다.")
        else :
            print("성공적으로 삭제하였습니다. ")
        cur.close()
        return result

In [6]:
from math import  *
from statistics import *

class Management:
    def __init__(self):
        self.maria = maria()
    def M_select(self):
        data = self.maria.selectdb()
        st = []
        if (data):
            for student in data:
                stp = Student()
                stp.name = student[0]
                stp.kor = student[1]
                stp.eng = student[2]
                stp.mat = student[3]
                st.append(stp)                    
            # studdata=Student((data))
            print(maria.schoolname + "  성적 계산표 ")
            print('%5s %5s %6s %6s %6s %6s  %6s %6s' % ('번 호', '이 름', '국 어',
                    '수 학', '영  어', '총 점', '평 균', '학 점' ))      
            for stud in st:
                print( stud)                     
        else :
            print("데이터가 없습니다. . ")
           
    def M_insert(self):
        stu = Student()
        stu.inputData()
        self.maria.insertdb(stu.name, stu.kor, stu.mat, stu.eng)
        
    #def M_calculate(self):
        #stu = Student()
        #stu.calc_total_average()
        
    def M_update(self):
        name = input("검색할 이름을 입력하시요")
        replace_name = input("수정할 이름을 입력하시요")
        replace_kor = eval(input("국어 점수를 입력하시오"))
        replace_mat = eval(input("수학 점수를 입력하시오"))
        replace_eng = eval(input("영어 점수를 입력하시오"))
        result=self.maria.updatedb(name, replace_name, replace_kor, replace_mat, replace_eng)  
    def M_delete(self):       
        name = input("검색할 이름을 입력하시요")
        self.maria.deletedb(name)     

In [ ]:
import sys
class Menu():
    def __init__(self):
        self.manage = Management()
       
    def work(self):
        menu = [ "전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)"]
        manage = Management()
        while 1:
            sel = input(menu)
            if  sel == "1":
                manage.M_select()
                continue
            elif sel == "2":
                manage.M_insert()
                continue       
            elif sel =="3":
                manage.M_update()
                continue
            elif sel =="4":
                manage.M_delete()
            elif sel =="9":
                del(manage)
                sys.exit()
            else :
                print("잘못된 입력입니다. 다시입력 하시요")
                continue
           
menu = Menu()
menu.work() 

테이블이 있습니다. 
테이블이 있습니다. 
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']1
에이콘 아카데미  성적 계산표 
  번 호   이 름    국 어    수 학   영  어    총 점     평 균    학 점
    0     1    김하나     98    100       0       0 
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']2
이름을 입력하세요이둘둘
국어 점수:69
수학 점수:92
영어 점수:72
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']1
에이콘 아카데미  성적 계산표 
  번 호   이 름    국 어    수 학   영  어    총 점     평 균    학 점
    0     1    김하나     98    100       0       0 
    0     2    이둘둘     92     69       0       0 
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']1
에이콘 아카데미  성적 계산표 
  번 호   이 름    국 어    수 학   영  어    총 점     평 균    학 점
    0     1    김하나     98    100       0       0 
    0     2    이둘둘     92     69       0       0 
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']2
이름을 입력하세요박세엣
국어 점수:87
수학 점수:49
영어 점수:78
['전체검색(1), 입력(2), 수정(3), 삭제(4), 종료(9)']1
에이콘 아카데미  성적 계산표 
  번 호   이 름    국 어    수 학   영  어    총 점     평 균    학 점
    0     1    김하나     98    100       0       0 
    0     2    이둘둘     92     69       0       0 
    0   

In [ ]:
# 일요일 밤 12시까지 과제 제출 (메일로) (CRUD 기능을 갖춘 프로그램)